In [68]:
import pandas as pd
from openai import OpenAI
import logging
import time

In [69]:
# Set up logging for debugging and create a separate log file
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', filename='error_log.txt', filemode='w')

#### 1. Read Data from CSV File

In [70]:
def read_csv(file_path):
    """
    Reads data from a CSV file into a pandas DataFrame.

    Args:
        file_path (str): The path to the CSV file.

    Returns:
        DataFrame: The data read from the CSV file.
    """
    try:
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)
        logging.info("Data read successfully from CSV file.")
        return df
    except Exception as e:
        # Log an error message if reading fails
        logging.error(f"Error reading CSV file: {e}")
        raise

#### 2.1. Get OpenAI API Key

In [71]:
def get_openai_api_key(file_path):
    """
    Reads the OpenAI API key from a file.

    Args:
        file_path (str): The path to the file containing the API key.

    Returns:
        str: The API key.
    """
    with open(file_path, 'r') as file:
        api_key = file.read().strip()
    return api_key

#### 2.2. Process Data with ChatGPT

In [72]:
def generate_calculations(df, api_key):
    """
    Sends a prompt to ChatGPT to generate cleaning and formatting calculations.

    Args:
        df (DataFrame): The DataFrame containing the data to be processed.
        api_key (str): The OpenAI API key.

    Returns:
        str: The cleaning and formatting calculations generated by ChatGPT.
    """
    # Get column names and data types
    columns_info = df.dtypes.to_dict()
    columns_prompt = "\n".join([f"{col}: {dtype}" for col, dtype in columns_info.items()])

    # Construct the prompt
    prompt = f"""
    The following DataFrame has the columns and data types:
    {columns_prompt}
    
    Generate Python code to clean and format this DataFrame. The code should:
    1. Standardize the format for each column, making it consistent and readable.
    2. Fill missing values with "null".
    3. Maintain the same column names in the output DataFrame.
    4. Return the cleaned DataFrame.

    Provide only the Python code for this task.
    """

    client = openai.OpenAI(api_key=api_key)
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    
    calculations = response.choices[0].message.content.strip()
    
    # Log the generated calculations
    logging.info(f"Generated Calculations: {calculations}")
    
    # Validate the calculations
    try:
        exec(calculations, globals(), locals())
        logging.info("Calculations are syntactically correct.")
    except SyntaxError as e:
        logging.error(f"Syntax error in generated calculations: {e}")
        raise
    
    return calculations

# Ensure the script is run as a main program
if __name__ == "__main__":
    main()


#### 3. Create Calculated Fields

In [73]:
def apply_calculations(df, calculations):
    """
    Applies the cleaning and formatting calculations to the DataFrame.

    Args:
        df (DataFrame): The DataFrame to be processed.
        calculations (str): The cleaning and formatting calculations.

    Returns:
        DataFrame: The processed DataFrame.
    """
    try:
        exec(calculations, globals(), locals())
        logging.info("Calculated fields applied successfully.")
        return df
    except SyntaxError as e:
        logging.error(f"Error applying calculations: {e}")
        raise

#### 4. Log and Flag Errors

In [74]:
def log_and_flag_errors(df):
    """
    Logs and flags errors in the DataFrame.

    Args:
        df (DataFrame): The DataFrame to be processed.

    Returns:
        DataFrame: The DataFrame with errors logged and flagged.
    """
    try:
        # Define a condition to identify errors (this is a placeholder)
        def some_condition(row):
            # Example condition: check if any cell is empty
            return row.isnull().any()

        # Apply the condition to flag errors
        df['Error_Flag'] = df.apply(lambda row: 'Invalid' if some_condition(row) else 'Valid', axis=1)
        
        # Log the rows with errors to a separate log file
        errors = df[df['Error_Flag'] == 'Invalid']
        errors.to_csv('errors_log.csv', index=False)
        
        logging.info("Errors logged and flagged successfully.")
        return df
    except Exception as e:
        # Log an error message if logging and flagging errors fails
        logging.error(f"Error logging and flagging errors: {e}")
        raise

#### 5. Prepare Data for Further Analysis

In [75]:
def save_cleaned_data(df, output_file_path):
    """
    Saves the cleaned data to a CSV file.

    Args:
        df (DataFrame): The DataFrame containing the cleaned data.
        output_file_path (str): The path to the output CSV file.
    """
    try:
        # Save the DataFrame to a CSV file
        df.to_csv(output_file_path, index=False)
        logging.info("Cleaned data saved successfully.")
    except Exception as e:
        # Log an error message if saving cleaned data fails
        logging.error(f"Error saving cleaned data: {e}")
        raise

#### Main

In [76]:
def main():
    """
    Main function to execute the data cleaning and formatting process.
    """
    try:
        input_file_path = 'Tableau/ISP/Summer 2024 Internship Stipend Program (Responses) - Form Responses 2.csv'
        output_file_path = 'Tableau/ISP/cleaned_data.csv'
        api_key_file_path = 'Tableau/ISP/openai_api_key.txt'

        # Read the CSV file
        df = read_csv(input_file_path)
        
        # Read the OpenAI API key from the file
        api_key = get_openai_api_key(api_key_file_path)
        
        # Generate calculations with ChatGPT using the API key
        calculations = generate_calculations(df, api_key)
        
        # Apply calculations
        df = apply_calculations(df, calculations)
        
        # Log and flag errors
        df = log_and_flag_errors(df)
        
        # Save the cleaned data
        save_cleaned_data(df, output_file_path)
        
        logging.info("Process completed successfully.")
    except Exception as e:
        logging.error(f"Process failed: {e}")

# Ensure the script is run as a main program
if __name__ == "__main__":
    main()